In [1]:
import qmcpy as qp
import numpy as np
import scipy.stats
from tabulate import tabulate
#import pandas as pd
import time
#from matplotlib import pyplot
#pyplot.style.use('../qmcpy/qmcpy.mplstyle')
#colors = pyplot.rcParams['axes.prop_cycle'].by_key()['color']
print("ok")

ok


We generated our points in $d$ dimensions and generated $n$ samples such that $\boldsymbol{x}_i \in [0,1]^d$ for $i = 1, 2, 3, ..., n$. Let matrix $\boldsymbol{X}$ be a $n \times d$ matrix such that
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_1\\
\boldsymbol{x}_2\\
\boldsymbol{x}_3\\
...\\
\boldsymbol{x}_n\\
\end{bmatrix}
$$
which can be rewritten as
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_{(1,1)} & \boldsymbol{x}_{(1,2)} & \boldsymbol{x}_{(1,3)} & ... & \boldsymbol{x}_{(1,d)}\\
\boldsymbol{x}_{(2,1)} & \boldsymbol{x}_{(2,2)} & \boldsymbol{x}_{(2,3)} & ... & \boldsymbol{x}_{(2,d)}\\
\boldsymbol{x}_{(3,1)} & \boldsymbol{x}_{(3,2)} & \boldsymbol{x}_{(3,3)} & ... & \boldsymbol{x}_{(3,d)}\\
... & ... & ... & ... & ...\\
\boldsymbol{x}_{(n,1)} & \boldsymbol{x}_{(n,2)} & \boldsymbol{x}_{(n,3)} & ... & \boldsymbol{x}_{(n,d)}\\
\end{bmatrix}
$$

We want to find our discrepancy in star discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
Note, the kernel for star discrepancy is $K(\boldsymbol{x}_i,\boldsymbol{x}_j) = \prod_{k=1}^d [2 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k})]$. Also, $\int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d$ and $\frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{3 - \boldsymbol{x}_{i_k}^2}{2})$. For the sake of program let,
$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{3 - \boldsymbol{x}_{i_k}^2}{2})
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d [2 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k})]
$$
So, $D(\boldsymbol{X}) = \sqrt{A - B + C}$. We calculate

In [2]:
def star_disc(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = (3-x**2)/2
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - np.maximum(c_1, y)
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in L2star discrepancy. The equation to find the discrepancy is the same except we choose a different equation for the kernel. Which our kernel is $K(x,y) = \prod_{k=1}^d (1 - \max(x_k,y_k))$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (1/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2}) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k}))
$$
$$
D(\boldsymbol{X}) = \sqrt{(1/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2}) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(\boldsymbol{x}_{i_k}, \boldsymbol{x}_{j_k}))}
$$
So now let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{1}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \prod_{k=1}^d (\frac{1 - \boldsymbol{x}_{i_k}^2}{2})
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 - \max(x_k,y_k))
$$
So, $D(\boldsymbol{X}) = \sqrt{A - B + C}$. We calculate

In [3]:
def L2star_disc(x):
    n, d = x.shape

    A = (1/3)**d

    b_1 = (1 - x**2)/2
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 1 - np.maximum(c_1, y)
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in centered discrepancy. We will choose our kernel to be $K(x,y) = \prod_{k=1}^d (1 + .5|\boldsymbol{x}_k - .5| + .5|\boldsymbol{y}_k - .5| - .5 |\boldsymbol{x}_k - \boldsymbol{y}_k|)$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (13/12)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
$$
D(\boldsymbol{X}) = \sqrt{(13/12)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)}
$$
Let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{13}{12})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| - .5 (\boldsymbol{x}_{i_k} -.5)^2)
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (1 + .5|\boldsymbol{x}_{i_k} - .5| + .5|\boldsymbol{x}_{j_k} - .5| - .5 |\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
Of course $D(\boldsymbol{X}) = \sqrt{A - B + C}$. So now we calculate,

In [4]:
def center_disc(x):
    n, d = x.shape

    A = (13/12)**d

    b_1 = 1 + (.5*abs(x - .5)) - (.5*((x -.5)**2))
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 1 + (.5*abs(c_1 - .5)) + (.5*abs(y - .5)) - (.5*abs(c_1 - y))
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

We want to find our discrepancy in symmetric discrepancy. We will choose our kernel to be $K(x,y) = \prod_{k=1}^d (2 - 2|\boldsymbol{x}_k - \boldsymbol{y}_k|)$. And so to get our discrepancy we must calculate
$$
[D(\boldsymbol{X}) ]^2= \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy - \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
$$
[D(\boldsymbol{X}) ]^2= (4/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
$$
D(\boldsymbol{X}) = \sqrt{(4/3)^d - \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2) + \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)}
$$
Let A, B, and C be calculated as

$$
A = \int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy = (\frac{4}{3})^d
$$
$$
B = \frac{2}{n} \sum_{i=1}^n \int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx = \frac{2}{n} \sum_{i=1}^n \prod_{k=1}^d (1 + 2\boldsymbol{x}_{i_k} - 2 \boldsymbol{x}_{i_k}^2)
$$
$$
C = \frac{1}{n^2} \sum_{i,j = 1}^n \prod_{k=1}^d (2 - 2|\boldsymbol{x}_{i_k} - \boldsymbol{x}_{j_k}|)
$$
Of course $D(\boldsymbol{X}) = \sqrt{A - B + C}$. So now we calculate,

In [5]:
def symmetric_disc(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = 1 + 2*x - (2*(x**2))
    b_2 = b_1.prod(axis=1)
    B = 2*np.mean(b_2)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - (2*abs(c_1 - y))
    c_3 = c_2.prod(axis=2)
    c_4 = np.mean(c_3)
    C = np.mean(c_4)

    disc = (A - B + C)**.5
    return disc

This and next cell are play cells. So the L2-star discrepancy works just as accurately as the scipy discrepancy; however, confused on the centered discrepancy because in my code I square root to get $D(\boldsymbol{X})$. However in scipy that doesn't seem to be the case. To prove it. I printed out the following outputs. So for the centered discrepancy, this would mean that either I was not suppose to square root or scipy forgot to square root.

In [6]:
#L2-star discrepancy
#Fix the bug in this program
d = 1
n = 4000
A = qp.IIDStdUniform(dimension=d)
X = A.gen_samples(n)
#X = np.array([[.5, .5]])
#weight = np.arange(1, d+1, dtype=float)**(-2)

#print(weight)

#weight = np.array([3, 2, 1])
#print(qp.discrepancy(SY, X))
#print(qp.discrepancy('Sy', X))

#print(qp.discrepancy('S', X))
#print(qp.discrepancy('S', X, 1))
d_1 = 3
#print(qp.discrepancy('S', X, weight))
print(X[3990:5000, :])
#print(weight[:d_1])
start = time.time()
p = qp.discrepancy('WA', X)
end = time.time() - start
print(p)
print(end)
#print(scipy.stats.qmc.discrepancy(X, method='WD'))
#print(np.sqrt(scipy.stats.qmc.discrepancy(X, method='WD')))

[[0.67787599]
 [0.81748144]
 [0.4473602 ]
 [0.40136272]
 [0.40669182]
 [0.12938863]
 [0.28688491]
 [0.06038174]
 [0.74906091]
 [0.00421478]]
0.011037985325410987
0.5980393886566162


In [9]:
#Example for using a discrepancy function given by the user for QMCSoftware's discrepancy function
#In this case, we are using Symmetric discrepancy function.

d = 3
A = qp.IIDStdUniform(dimension=d)
X = A.gen_samples(n=2)

def SY(x):
    n, d = x.shape

    A = (4/3)**d

    b_1 = 1 + 2*x - (2*(x**2))
    B = b_1.prod(axis=1)

    c_1 = np.zeros((n,n,d)) + x
    y = np.resize(x, (n, 1, d))
    c_2 = 2 - (2*abs(c_1 - y))
    C = c_2.prod(axis=2)
    return A, B, C

print(qp.discrepancy(SY, X))

1.367048639728414


In [14]:
P1_star = np.array([[1/7, 4/7], [2/7, 1/7], [3/7, 5/7], [4/7, 2/7], [5/7, 6/7], [6/7, 3/7], [1, 1]]) - (1/14)
P2_star = np.array([[1/7, 5/7], [2/7, 2/7], [3/7, 1], [4/7, 4/7], [5/7, 1/7], [6/7, 6/7], [1, 3/7]]) - (1/14)
P3_star = np.array([[1/7, 6/7], [2/7, 1/7], [3/7, 3/7], [4/7, 5/7], [5/7, 1], [6/7, 2/7], [1, 4/7]]) - (1/14)
P4_star = np.array([[1/7, 3/7], [2/7, 6/7], [3/7, 1/7], [4/7, 4/7], [5/7, 1], [6/7, 2/7], [1, 5/7]]) - (1/14)
print(' ')
p1_star_qmcpy_wa = round(qp.discrepancy('Wrap around', P1_star),6)
p1_star_sq_scipy_wa = round(np.sqrt(scipy.stats.qmc.discrepancy(P1_star, method='WD')),6)
p1_star_scipy_wa = round(scipy.stats.qmc.discrepancy(P1_star, method='WD'), 6)
p1_star_qmcpy_c = round(qp.discrepancy('Centered', P1_star), 6)
p1_star_sq_scipy_c = round(np.sqrt(scipy.stats.qmc.discrepancy(P1_star, method='CD')), 6)
p1_star_scipy_c = round(scipy.stats.qmc.discrepancy(P1_star, method='CD'),6)
p1_star_qmcpy_m = round(qp.discrepancy('Mixture', P1_star), 6)
p1_star_sq_scipy_m = round(np.sqrt(scipy.stats.qmc.discrepancy(P1_star, method='MD')),6)
p1_star_scipy_m = round(scipy.stats.qmc.discrepancy(P1_star, method='MD'),6)

p2_star_qmcpy_wa = round(qp.discrepancy('Wrap around', P2_star),6)
p2_star_sq_scipy_wa = round(np.sqrt(scipy.stats.qmc.discrepancy(P2_star, method='WD')),6)
p2_star_scipy_wa = round(scipy.stats.qmc.discrepancy(P2_star, method='WD'),6)
p2_star_qmcpy_c = round(qp.discrepancy('Centered', P2_star),6)
p2_star_sq_scipy_c = round(np.sqrt(scipy.stats.qmc.discrepancy(P2_star, method='CD')),6)
p2_star_scipy_c = round(scipy.stats.qmc.discrepancy(P2_star, method='CD'),6)
p2_star_qmcpy_m = round(qp.discrepancy('Mixture', P2_star),6)
p2_star_sq_scipy_m = round(np.sqrt(scipy.stats.qmc.discrepancy(P2_star, method='MD')),6)
p2_star_scipy_m = round(scipy.stats.qmc.discrepancy(P2_star, method='MD'),6)

p3_star_qmcpy_wa = round(qp.discrepancy('Wrap around', P3_star),6)
p3_star_sq_scipy_wa = round(np.sqrt(scipy.stats.qmc.discrepancy(P3_star, method='WD')),6)
p3_star_scipy_wa = round(scipy.stats.qmc.discrepancy(P3_star, method='WD'),6)
p3_star_qmcpy_c = round(qp.discrepancy('Centered', P3_star),6)
p3_star_sq_scipy_c = round(np.sqrt(scipy.stats.qmc.discrepancy(P3_star, method='CD')),6)
p3_star_scipy_c = round(scipy.stats.qmc.discrepancy(P3_star, method='CD'),6)
p3_star_qmcpy_m = round(qp.discrepancy('Mixture', P3_star),6)
p3_star_sq_scipy_m = round(np.sqrt(scipy.stats.qmc.discrepancy(P3_star, method='MD')),6)
p3_star_scipy_m = round(scipy.stats.qmc.discrepancy(P3_star, method='MD'),6)

p4_star_qmcpy_wa = round(qp.discrepancy('Wrap around', P4_star),6)
p4_star_sq_scipy_wa = round(np.sqrt(scipy.stats.qmc.discrepancy(P4_star, method='WD')),6)
p4_star_scipy_wa = round(scipy.stats.qmc.discrepancy(P4_star, method='WD'),6)
p4_star_qmcpy_c = round(qp.discrepancy('Centered', P4_star),6)
p4_star_sq_scipy_c = round(np.sqrt(scipy.stats.qmc.discrepancy(P4_star, method='CD')),6)
p4_star_scipy_c = round(scipy.stats.qmc.discrepancy(P4_star, method='CD'),6)
p4_star_qmcpy_m = round(qp.discrepancy('Mixture', P4_star),6)
p4_star_sq_scipy_m = round(np.sqrt(scipy.stats.qmc.discrepancy(P4_star, method='MD')),6)
p4_star_scipy_m = round(scipy.stats.qmc.discrepancy(P4_star, method='MD'),6)

print('QMCPy')
mydata = [['No.', 'P_1^*', 'P_2^*', 'P_3^*', 'P_4^*'], 
          ['WD', p1_star_qmcpy_wa, p2_star_qmcpy_wa, p3_star_qmcpy_wa, p4_star_qmcpy_wa],
          ['CD', p1_star_qmcpy_c, p2_star_qmcpy_c, p3_star_qmcpy_c, p4_star_qmcpy_c],
          ['MD', p1_star_qmcpy_m, p2_star_qmcpy_m, p3_star_qmcpy_m, p4_star_qmcpy_m]
         ]
print(tabulate(mydata, tablefmt="grid"))
print(' ')
print('Scipy Square root')
mydata = [['No.', 'P_1^*', 'P_2^*', 'P_3^*', 'P_4^*'], 
          ['WD', p1_star_sq_scipy_wa, p2_star_sq_scipy_wa, p3_star_sq_scipy_wa, p4_star_sq_scipy_wa],
          ['CD', p1_star_sq_scipy_c, p2_star_sq_scipy_c, p3_star_sq_scipy_c, p4_star_sq_scipy_c],
          ['MD', p1_star_sq_scipy_m, p2_star_sq_scipy_m, p3_star_sq_scipy_m, p4_star_sq_scipy_m]
         ]
print(tabulate(mydata, tablefmt="grid"))
print(' ')
print('Scipy')
mydata = [['No.', 'P_1^*', 'P_2^*', 'P_3^*', 'P_4^*'], 
          ['WD', p1_star_scipy_wa, p2_star_scipy_wa, p3_star_scipy_wa, p4_star_scipy_wa],
          ['CD', p1_star_scipy_c, p2_star_scipy_c, p3_star_scipy_c, p4_star_scipy_c],
          ['MD', p1_star_scipy_m, p2_star_scipy_m, p3_star_scipy_m, p4_star_scipy_m]
         ]
print(tabulate(mydata, tablefmt="grid"))

 
QMCPy
+-----+----------+----------+----------+----------+
| No. | P_1^*    | P_2^*    | P_3^*    | P_4^*    |
+-----+----------+----------+----------+----------+
| WD  | 0.111417 | 0.111722 | 0.111417 | 0.111722 |
+-----+----------+----------+----------+----------+
| CD  | 0.088582 | 0.076314 | 0.078619 | 0.076314 |
+-----+----------+----------+----------+----------+
| MD  | 0.107926 | 0.104505 | 0.105979 | 0.104505 |
+-----+----------+----------+----------+----------+
 
Scipy Square root
+-----+----------+----------+----------+----------+
| No. | P_1^*    | P_2^*    | P_3^*    | P_4^*    |
+-----+----------+----------+----------+----------+
| WD  | 0.111417 | 0.111722 | 0.111417 | 0.111722 |
+-----+----------+----------+----------+----------+
| CD  | 0.088582 | 0.076314 | 0.078619 | 0.076314 |
+-----+----------+----------+----------+----------+
| MD  | 0.107926 | 0.104505 | 0.105979 | 0.104505 |
+-----+----------+----------+----------+----------+
 
Scipy
+-----+----------+----------

In [24]:
P_1 = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3],
               [1, 1, 2, 2, 2, 3, 3, 3, 1, 1, 1, 2, 2, 2, 3, 3, 1, 1, 1, 2, 2, 3, 3, 3],
               [1, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3, 2, 3, 1, 2, 3]]).T / 3 -(1/6)

                
P_2 = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2 ,2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3],
               [1, 1, 1, 2, 2, 3, 3, 3, 1, 1, 2, 2, 2, 2, 3, 3, 1, 1, 1, 2, 2, 3, 3, 3],
               [1, 2, 3, 1, 3, 1, 2, 3, 1, 3, 2, 2, 2, 2, 1, 3, 1, 2, 3, 1, 3, 1, 2, 3]]).T / 3 - (1/6)
print('P_1: Centered discrepancy')
print('QMCPy CD^2(P_1): ' + str(qp.discrepancy('Centered', P_1)**2))
print('Scipy CD^2(P_1): ' + str(scipy.stats.qmc.discrepancy(P_1, method='CD')))
print(' ')
print('P_2: Centered discrepancy')
print('QMCPy CD^2(P_2): ' + str(qp.discrepancy('Centered', P_2)**2))
print('Scipy CD^2(P_2): ' + str(scipy.stats.qmc.discrepancy(P_2, method='CD')))
print(' ')
print('P_1: Wrap around discrepancy')
print('QMCPy WD^2(P_1): ' + str(qp.discrepancy('Wrap around', P_1)**2))
print('Scipy WD^2(P_1): ' + str(scipy.stats.qmc.discrepancy(P_1, method='WD')))
print(' ')
print('P_2: Wrap around discrepancy')
print('QMCPy WD^2(P_2): ' + str(qp.discrepancy('Wrap around', P_2)**2))
print('Scipy WD^2(P_2): ' + str(scipy.stats.qmc.discrepancy(P_2, method='WD')))
###Add square to each of these discrepancies

P_1: Centered discrepancy
QMCPy CD^2(P_1): 0.03277892089620438
Scipy CD^2(P_1): 0.032778920896208597
 
P_2: Centered discrepancy
QMCPy CD^2(P_2): 0.03258601966163677
Scipy CD^2(P_2): 0.032586019661641874
 
P_1: Wrap around discrepancy
QMCPy WD^2(P_1): 0.10085162322816643
Scipy WD^2(P_1): 0.10085162322816599
 
P_2: Wrap around discrepancy
QMCPy WD^2(P_2): 0.10173754000914494
Scipy WD^2(P_2): 0.10173754000914359
 


For the solutions from Scipy, these solutions are correct. Since you are trying to find $CD(\mathscr{P}_1)^2$, $CD(\mathscr{P}_2)^2$, $WD(\mathscr{P}_1)^2$, $WD(\mathscr{P}_2)^2$. And these solutions were calculated correctly. However, there is a typo in Fang's paper on page 292.

Example 1, 3 in Dr. Fang's paper
Example 1 is the Table with P1 and P2
Example 3 is the Table with the error where they didn't take the square root to find WD instead of WD^2
This is why we agree in Example 1, and not example 3
His discrepancy is agreeing with the typo.

In [35]:
d = 500
A = qp.IIDStdUniform(dimension=d)
X = A.gen_samples(n=1024)
print(X)

weight = np.arange(1, d+1, dtype=float)**(-2)
#Do something for when d is higher than and just take the first d+1 values.

def L2(x,weight):
    n, d = x.shape
    Y = np.resize(x, (n, 1, d))
    X_expanded = np.resize(x, (1, n, d))
    
    A = (1 + (weight/3)).prod(axis=0)

    B = ((1 + (weight*(1 - x**2)/2))).prod(axis=1)

    C = (1 + weight*(1 - np.maximum(X_expanded, Y))).prod(axis=2)
    return A, B, C

def L(x):
    n, d = x.shape
    Y = np.resize(x, (n, 1, d))
    X_expanded = np.resize(x, (1, n, d))
    
    A = (4/3)**d

    B = (1 + ((1 - x**2)/2)).prod(axis=1)

    C = (2 - np.maximum(X_expanded, Y)).prod(axis=2)
    return A, B, C


print(qp.discrepancy(L2, X))
print(
print(qp.discrepancy('S', X, weight))
print(qp.discrepancy(L, X))
print(qp.discrepancy('S', X))

[[0.19425817 0.64145901 0.67391544 ... 0.78545788 0.67589707 0.71817023]
 [0.35939075 0.00376301 0.72943479 ... 0.43663096 0.12051892 0.21074073]
 [0.10968795 0.50023891 0.81302315 ... 0.87861972 0.73683258 0.73199218]
 ...
 [0.88413185 0.80251752 0.18365481 ... 0.14243569 0.03400094 0.94343327]
 [0.62082038 0.70017901 0.58245673 ... 0.74203103 0.0975401  0.7142105 ]
 [0.92617888 0.23708961 0.95689551 ... 0.43350336 0.517326   0.21372566]]


KeyboardInterrupt: 

In [16]:
d = 3
A = qp.IIDStdUniform(dimension=d)
X = A.gen_samples(n=2)
print('wrap around')
print('QMCSoftware:          ' + str(qp.discrepancy('WA', X)))
print('Square root of Scipy: ' + str(np.sqrt(scipy.stats.qmc.discrepancy(X, method='WD'))))
print('Scipy:                ' + str(scipy.stats.qmc.discrepancy(X, method='WD')))

print('centered')
print('QMCSoftware:          ' + str(qp.discrepancy('C', X)))
print('Square root of Scipy: ' + str(np.sqrt(scipy.stats.qmc.discrepancy(X, method='CD'))))
print('Scipy:                ' + str(scipy.stats.qmc.discrepancy(X, method='CD')))

print('mixture')
print('QMCSoftware:          ' + str(qp.discrepancy('M', X)))
print('Square root of Scipy: ' + str(np.sqrt(scipy.stats.qmc.discrepancy(X, method='MD'))))
print('Scipy:                ' + str(scipy.stats.qmc.discrepancy(X, method='MD')))

wrap around
QMCSoftware:          0.6477286434405766
Square root of Scipy: 0.6477286434405762
Scipy:                0.4195523955333691
centered
QMCSoftware:          0.5696302657109088
Square root of Scipy: 0.5696302657109088
Scipy:                0.32447863961388057
mixture
QMCSoftware:          0.7566786568383785
Square root of Scipy: 0.7566786568383785
Scipy:                0.5725625897147326


In [9]:
A = qp.IIDStdUniform(dimension=3)
X = A.gen_samples(n=4)
#print(np.repeat(3, 4))
#x = np.array([[1,2],[3,4]])
#print(np.repeat(x, 2))
#print(np.repeat(x, 3, axis=1))
#print(np.repeat(x, [1, 2], axis=0))
n, d = X.shape
print(X)
X_expanded = np.repeat(X[np.newaxis, :, :], n, axis= 0)
print(X_expanded)
print(X_expanded[1, 1, 2])
print(X_expanded.shape)
Y_expanded = np.zeros((n,n,d)) + X
print(Y_expanded - X_expanded)

[[0.40898697 0.54669356 0.1618548 ]
 [0.50089173 0.90893622 0.23370881]
 [0.80956436 0.29776616 0.38910199]
 [0.45827124 0.17344588 0.86071439]]
[[[0.40898697 0.54669356 0.1618548 ]
  [0.50089173 0.90893622 0.23370881]
  [0.80956436 0.29776616 0.38910199]
  [0.45827124 0.17344588 0.86071439]]

 [[0.40898697 0.54669356 0.1618548 ]
  [0.50089173 0.90893622 0.23370881]
  [0.80956436 0.29776616 0.38910199]
  [0.45827124 0.17344588 0.86071439]]

 [[0.40898697 0.54669356 0.1618548 ]
  [0.50089173 0.90893622 0.23370881]
  [0.80956436 0.29776616 0.38910199]
  [0.45827124 0.17344588 0.86071439]]

 [[0.40898697 0.54669356 0.1618548 ]
  [0.50089173 0.90893622 0.23370881]
  [0.80956436 0.29776616 0.38910199]
  [0.45827124 0.17344588 0.86071439]]]
0.23370880692199503
(4, 4, 3)
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [9]:
weight = np.array([1, 2, 3])
weight2 = np.array([1, 1, 1])
print(weight*weight2)

[1 2 3]


In [11]:
y = np.ones(3)

In [10]:
x = np.array([1, 2, 3])
y = np.array([[1], [2], [3]])
print(np.maximum(x,y))


[[1 2 3]
 [2 2 3]
 [3 3 3]]


In [10]:
np.sqrt((1/3)**2 - (2*9/64) + .25)

0.2825970826302195

In [ ]:
print(scipy.stats.qmc.discrepancy(X, method='L2-star'))